In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_csv("air_quality_dataset.csv") 
print(df.info()) #check info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Rank     2164 non-null   int64  
 1   City     2164 non-null   object 
 2   Country  2164 non-null   object 
 3   2023     2164 non-null   float64
 4   Jan      2164 non-null   object 
 5   Feb      2164 non-null   object 
 6   Mar      2164 non-null   object 
 7   Apr      2164 non-null   object 
 8   May      2164 non-null   object 
 9   Jun      2164 non-null   object 
 10  Jul      2164 non-null   object 
 11  Aug      2164 non-null   object 
 12  Sep      2164 non-null   object 
 13  Oct      2164 non-null   object 
 14  Nov      2164 non-null   object 
 15  Dec      2164 non-null   object 
dtypes: float64(1), int64(1), object(14)
memory usage: 270.6+ KB
None


Data types should be similar so we need to change to float

In [4]:
for col in df.columns[-12:]:
    df[col] = pd.to_numeric(df[col],errors="coerce")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Rank     2164 non-null   int64  
 1   City     2164 non-null   object 
 2   Country  2164 non-null   object 
 3   2023     2164 non-null   float64
 4   Jan      2077 non-null   float64
 5   Feb      2098 non-null   float64
 6   Mar      2131 non-null   float64
 7   Apr      2155 non-null   float64
 8   May      2106 non-null   float64
 9   Jun      2158 non-null   float64
 10  Jul      2160 non-null   float64
 11  Aug      2153 non-null   float64
 12  Sep      2153 non-null   float64
 13  Oct      2133 non-null   float64
 14  Nov      2073 non-null   float64
 15  Dec      2113 non-null   float64
dtypes: float64(13), int64(1), object(2)
memory usage: 270.6+ KB


In [5]:
## check missing values by columns
df.isnull().sum(axis=0).sort_values(ascending=False)

Nov        91
Jan        87
Feb        66
May        58
Dec        51
Mar        33
Oct        31
Aug        11
Sep        11
Apr         9
Jun         6
Jul         4
Rank        0
City        0
Country     0
2023        0
dtype: int64

In [6]:
np.sum(df.isnull().sum(axis=1) > 0)

np.int64(270)

In [ ]:
print(df.isnull().sum().sum())

458


270 out of 2164 rows (12.48%) contain at least one missing value. And total amount is 458 missed value